# Modeling attenuation in 3-layers model using SPECFEM2D 

In [ ]:
import sys

pwd = !echo ${PWD}
sys.path.append(pwd[0]+"/../../../code/local/bin")

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.ticker import FormatStrFormatter

import seppy

sep = seppy.sep()

rcParams['font.size'] = 8
rcParams['font.family'] = 'sans-serif'

# Create directories to store data and figures
!mkdir -p ../dat ../fig

datapath=pwd[0]+"/../dat/"
figpath=pwd[0]+"/../fig/"

In [ ]:
# source parameters
nt=8000 # number of time samples
dt=0.00005 # sampling rate (sec)
ot=0.0 # origin time
f0=150. # peak frequency (Hz)
shift=500 # shift wavelet (number of samples)

In [ ]:
# Generate a Butterworth source
!${PWD}/../../../code/local/bin/GENERATE_WAVELET.x nt=800 dt=0.001 type=butterworth low_cutoff=0.1 high_cutoff=0.4 half_order=1 phase=zero \
output=../dat/src0.H datapath=${PWD}/../dat/

# Resample the wavelet for SPECFEM2D
!${PWD}/../../../code/local/bin/RESAMPLE.x input=../dat/src0.H factor=20 output=../dat/src0.H.dense datapath=${PWD}/../dat/

In [ ]:
# read back the sources and window them
axes, data = sep.read_file(datapath+"src0.H")
src0 = data.reshape(axes.n,order='F')
src0 = src0[int((nt-shift)/10):int((nt-shift)/10)+int(nt/10)]

axes, data = sep.read_file(datapath+"src0.H.dense")
src0dense = data.reshape(axes.n,order='F')
src0dense = src0dense[int(nt-shift):int(nt-shift+nt)]

In [ ]:
# save to text file for SPECFEM2D
file = open(datapath+"src1.txt", 'w')
for i in range(nt):
    print("%.6f %.6f" %(i*dt+ot,src0dense[i]), file=file)
file.close()

# save to SEPlib
sep.write_file(datapath+"src1.H", np.transpose(src0dense[0:nt:10]), ds=np.array([10*dt]), os=np.array([0]), dpath=datapath)

In [ ]:
%%capture captured1
# run SPECFEM2D for elastic isotropic
!bash ../specfem2d/run1.sh

In [ ]:
vx=np.fromfile(datapath+"/OUTPUT_FILES/Ux_file_single_d.bin", dtype=np.float32)
vz=np.fromfile(datapath+"/OUTPUT_FILES/Uz_file_single_d.bin", dtype=np.float32)
vx=np.reshape(vx, (491,800))
vz=np.reshape(vz, (491,800))
sf1=np.zeros((2,491,800))
sf1[0,:,:] = vx
sf1[1,:,:] = vz
sep.write_file(datapath+"sf1.H", np.transpose(sf1), ds=np.array([0.0005,1,1]), os=np.array([0,0,0]), dpath=datapath)

In [ ]:
def QConversion(Qp, Qs, Vp, Vs):
    """Convert Qp and Qs to Qkappa and Qmu in 2D to properly parameterize Viscoelastic simulationn in SPECFEM2D"""

    invQp = 1./Qp
    invQs = 1./Qs

    invQkappa = (invQp - (Vs/Vp)**2 * invQs) / (1. - (Vs/Vp)**2)
    Qkappa = 1./invQkappa
    Qmu = Qs

    return Qkappa, Qmu

In [ ]:
print("For Qp=Qs=100 enter")
print(QConversion(100., 100., 3.7, 1.7))

print("For Qp=Qs=60 enter")
print(QConversion(60., 60., 3.7, 1.7))

print("For Qp=Qs=30 enter")
print(QConversion(30., 30., 3.7, 1.7))

In [ ]:
%%capture captured2
# run SPECFEM2D for viscoelastic isotropic, Qp=100, Qs=100
!bash ../specfem2d/run2.sh

In [ ]:
vx=np.fromfile(datapath+"/OUTPUT_FILES/Ux_file_single_d.bin", dtype=np.float32)
vz=np.fromfile(datapath+"/OUTPUT_FILES/Uz_file_single_d.bin", dtype=np.float32)
vx=np.reshape(vx, (491,800))
vz=np.reshape(vz, (491,800))
sf2=np.zeros((2,491,800))
sf2[0,:,:] = vx
sf2[1,:,:] = vz
sep.write_file(datapath+"sf2.H", np.transpose(sf2), ds=np.array([0.0005,1,1]), os=np.array([0,0,0]), dpath=datapath)

In [ ]:
%%capture captured3
# run SPECFEM2D for viscoelastic isotropic, Qp=60, Qs=60
!bash ../specfem2d/run3.sh

In [ ]:
vx=np.fromfile(datapath+"/OUTPUT_FILES/Ux_file_single_d.bin", dtype=np.float32)
vz=np.fromfile(datapath+"/OUTPUT_FILES/Uz_file_single_d.bin", dtype=np.float32)
vx=np.reshape(vx, (491,800))
vz=np.reshape(vz, (491,800))
sf3=np.zeros((2,491,800))
sf3[0,:,:] = vx
sf3[1,:,:] = vz
sep.write_file(datapath+"sf3.H", np.transpose(sf3), ds=np.array([0.0005,1,1]), os=np.array([0,0,0]), dpath=datapath)

In [ ]:
%%capture captured4
# run SPECFEM2D for viscoelastic isotropic, Qp=30, Qs=30
!bash ../specfem2d/run4.sh

In [ ]:
vx=np.fromfile(datapath+"/OUTPUT_FILES/Ux_file_single_d.bin", dtype=np.float32)
vz=np.fromfile(datapath+"/OUTPUT_FILES/Uz_file_single_d.bin", dtype=np.float32)
vx=np.reshape(vx, (491,800))
vz=np.reshape(vz, (491,800))
sf4=np.zeros((2,491,800))
sf4[0,:,:] = vx
sf4[1,:,:] = vz
sep.write_file(datapath+"sf4.H", np.transpose(sf4), ds=np.array([0.0005,1,1]), os=np.array([0,0,0]), dpath=datapath)

In [ ]:
# build the model to run simulation using my FD-SBP code for QC
nx=1201
nz=181
model=np.zeros((3,nx,nz))

vp=np.array([4.0,3.7,5.6])
vs=np.array([2.3,1.7,3.3])
ro=np.array([2.7,2.3,2.7])

mu = ro*vs**2
ka = ro*vp**2 - 4./3.*mu

ka1=2.0/(1.0/ka[0] +  1.0/ka[1])
ka2=2.0/(1.0/ka[1] +  1.0/ka[2])
mu1=2.0/(1.0/mu[0] +  1.0/mu[1])
mu2=2.0/(1.0/mu[1] +  1.0/mu[2])
ro1=0.5*(ro[0]+ro[1])
ro2=0.5*(ro[1]+ro[2])

# top layer (overburden)
model[0,:,0:84]=vp[0] # Vp in km/s
model[1,:,0:84]=vs[0] # Vs in km/s
model[2,:,0:84]=ro[0] # rho in g/cc

# middle layer (reservoir)
model[0,:,85:100]=vp[1]
model[1,:,85:100]=vs[1]
model[2,:,85:100]=ro[1]

# bottom layer (basement)
model[0,:,101:]=vp[2]
model[1,:,101:]=vs[2]
model[2,:,101:]=ro[2]

# at the two interfaces, use harmonic average for bulk and shear modulii
# and arithmetic average for density
model[0,:,84] = math.sqrt((ka1+mu1*4./3.)/ro1)
model[0,:,100] = math.sqrt((ka2+mu2*4./3.)/ro2)
model[1,:,84] = math.sqrt(mu1/ro1)
model[1,:,100] = math.sqrt(mu2/ro2)
model[2,:,84] = ro1
model[2,:,100] = ro2

In [ ]:
# save model to SEPlib
sep.write_file(datapath+"model.H", np.transpose(model), ds=np.array([0.001,0.001,1]), os=np.array([-0.04,-0.1,0]), dpath=datapath)

In [ ]:
# generate particle velocity data
!${PWD}/../../../code/local_gpu/bin/WE_MODELING.x source=../dat/src1.H model=../dat/model.H output=../dat/data.H datapath=${PWD}/../dat/ \
verbose=0 seismotype=0 mt=1 mxx=1 mzz=0.7 mxz=0 \
ns=1 sx0=0.25 sz0=0.055 nr=491 rx0=0.26 rz0=0.055 rxinc=0.001 \
bc_top=2 bc_bottom=2 bc_left=2 bc_right=2 taper_top=40 taper_bottom=40 taper_left=120 taper_right=40 taper_strength=0.1

In [ ]:
# FX-spectra
!${PWD}/../../../code/local/bin/FX.x < ../dat/sf1.H datapath=${PWD}/../dat/ output=../dat/sf1.H.fx
!${PWD}/../../../code/local/bin/FX.x < ../dat/sf2.H datapath=${PWD}/../dat/ output=../dat/sf2.H.fx
!${PWD}/../../../code/local/bin/FX.x < ../dat/sf3.H datapath=${PWD}/../dat/ output=../dat/sf3.H.fx
!${PWD}/../../../code/local/bin/FX.x < ../dat/sf4.H datapath=${PWD}/../dat/ output=../dat/sf4.H.fx
!${PWD}/../../../code/local/bin/FX.x < ../../ch5/dat/ch5st_data_group2.H ${PWD}/../dat/ output=../dat/das.H.fx

### Figures

In [ ]:
# load data
axes, sf1 = sep.read_file(datapath+"sf1.H")
sf1 = sf1.reshape(axes.n,order='F').T
sf1 = sf1[0]

axes, sf2 = sep.read_file(datapath+"sf2.H")
sf2 = sf2.reshape(axes.n,order='F').T
sf2 = sf2[0]

axes, sf3 = sep.read_file(datapath+"sf3.H")
sf3 = sf3.reshape(axes.n,order='F').T
sf3 = sf3[0]

axes, sf4 = sep.read_file(datapath+"sf4.H")
sf4 = sf4.reshape(axes.n,order='F').T
sf4 = sf4[0]

axes_sf_fx, sf1_fx = sep.read_file(datapath+"sf1.H.fx")
sf1_fx = sf1_fx.reshape(axes_sf_fx.n,order='F').T

axes_sf_fx, sf2_fx = sep.read_file(datapath+"sf2.H.fx")
sf2_fx = sf2_fx.reshape(axes_sf_fx.n,order='F').T

axes_sf_fx, sf3_fx = sep.read_file(datapath+"sf3.H.fx")
sf3_fx = sf3_fx.reshape(axes_sf_fx.n,order='F').T

axes_sf_fx, sf4_fx = sep.read_file(datapath+"sf4.H.fx")
sf4_fx = sf4_fx.reshape(axes_sf_fx.n,order='F').T

axes_fx, das_fx = sep.read_file(datapath+"das.H.fx")
das_fx = das_fx.reshape(axes_fx.n,order='F').T

In [ ]:
p=0.1
vmax=p*np.amax(sf1)

fig=plt.figure(figsize=(6.66, 6.66),dpi=300)

ax = plt.subplot(2,2,1)
plt.imshow(np.transpose(sf1),interpolation='sinc',aspect="auto",extent=[10,500,0.4,0],cmap='Greys',vmin=-vmax,vmax=vmax)
plt.ylabel('Time (s)')
plt.gca().set_xticklabels([])
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'a) $Q_P=Q_S=\infty$',loc='left')

ax = plt.subplot(2,2,2)
plt.imshow(np.transpose(sf2),interpolation='sinc',aspect="auto",extent=[10,500,0.4,0],cmap='Greys',vmin=-vmax,vmax=vmax)
plt.gca().set_xticklabels([])
plt.gca().set_yticklabels([])
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'b) $Q_P=Q_S=100$',loc='left')

ax = plt.subplot(2,2,3)
plt.imshow(np.transpose(sf3),interpolation='sinc',aspect="auto",extent=[10,500,0.4,0],cmap='Greys',vmin=-vmax,vmax=vmax)
plt.xlabel('Offset (m)')
plt.ylabel('Time (s)')
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'c) $Q_P=Q_S=60$',loc='left')

ax = plt.subplot(2,2,4)
plt.imshow(np.transpose(sf4),interpolation='sinc',aspect="auto",extent=[10,500,0.4,0],cmap='Greys',vmin=-vmax,vmax=vmax)
plt.xlabel('Offset (m)')
plt.gca().set_yticklabels([])
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'd) $Q_P=Q_S=30$',loc='left')

plt.tight_layout()
# plt.tight_layout(pad=0., w_pad=-2., h_pad=0.)

plt.savefig(figpath+'synthetic.png',bbox_inches='tight',format='png')

In [ ]:
def findMed(x,d):
    """Find the index value that splits the curve x in half"""
    n=x.shape[0]
    istart = 0
    iend = n
    while istart != iend:
        if np.trapz(x[:iend],dx=d) > 0.5:
            iend = int((istart + iend )/2)
        else:
            istart, iend = iend, int((iend + n )/2)
    
    return iend

In [ ]:
# synthetic data average spectra

nf=axes_sf_fx.n[0]
df=axes_sf_fx.d[0]
fmin=axes_sf_fx.o[0]
fmax=df*(nf-1) + fmin

xarray = np.linspace(0,(nf-1)*df,nf)

plt.figure(figsize=(6.66, 6.66),dpi=300)
plt.subplot(2,2,1)

spectrum1 = np.sum(sf1_fx[0,140:190,:],axis=0) # the first offset is 10 m
spectrum2 = np.sum(sf1_fx[0,440:490,:],axis=0)
area1 = np.trapz(spectrum1,dx=df)
area2 = np.trapz(spectrum2,dx=df)
spectrum1 /= area1
spectrum2 /= area2
med1 = xarray[findMed(spectrum1,df)]
med2 = xarray[findMed(spectrum2,df)]
ymax=max(np.max(spectrum1),np.max(spectrum2))
print("Near offset med freq - Far offset med freq = %.1f Hz" %(med1-med2))

plt.plot(xarray,spectrum1,color='b',linewidth=2,label="Near offsets")
plt.plot(xarray,spectrum2,color='r',linewidth=1,label="Far offsets")
plt.vlines(med1,0,ymax,colors='b',linestyles='--',linewidth=0.5)
plt.vlines(med2,0,ymax,colors='r',linestyles='--',linewidth=0.5)
plt.ylabel("Normalized amplitude")
plt.gca().set_xticklabels([])
plt.gca().set_yticks([])
plt.xlim([0,500])
plt.ylim([0,ymax])
plt.grid(color='k', linestyle='-', linewidth=0.2)
# plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.legend(loc='upper right',prop={'size': 6})
plt.title(r'a) $Q_P=Q_S=\infty$',loc='left')

plt.subplot(2,2,2)

spectrum1 = np.sum(sf2_fx[0,140:190,:],axis=0)
spectrum2 = np.sum(sf2_fx[0,440:490,:],axis=0)
area1 = np.trapz(spectrum1,dx=df)
area2 = np.trapz(spectrum2,dx=df)
spectrum1 /= area1
spectrum2 /= area2
med1 = xarray[findMed(spectrum1,df)]
med2 = xarray[findMed(spectrum2,df)]
ymax=max(np.max(spectrum1),np.max(spectrum2))
print("Near offset med freq - Far offset med freq = %.1f Hz" %(med1-med2))

plt.plot(xarray,spectrum1,color='b',linewidth=2,label="Near offsets")
plt.plot(xarray,spectrum2,color='r',linewidth=1,label="Far offsets")
plt.vlines(med1,0,ymax,colors='b',linestyles='--',linewidth=0.5)
plt.vlines(med2,0,ymax,colors='r',linestyles='--',linewidth=0.5)
plt.gca().set_xticklabels([])
plt.gca().set_yticks([])
plt.xlim([0,500])
plt.ylim([0,ymax])
plt.grid(color='k', linestyle='-', linewidth=0.2)
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.legend(loc='upper right',prop={'size': 6})
plt.title(r'b) $Q_P=Q_S=100$',loc='left')

plt.subplot(2,2,3)

spectrum1 = np.sum(sf3_fx[0,140:190,:],axis=0)
spectrum2 = np.sum(sf3_fx[0,440:490,:],axis=0)
area1 = np.trapz(spectrum1,dx=df)
area2 = np.trapz(spectrum2,dx=df)
spectrum1 /= area1
spectrum2 /= area2
med1 = xarray[findMed(spectrum1,df)]
med2 = xarray[findMed(spectrum2,df)]
ymax=max(np.max(spectrum1),np.max(spectrum2))
print("Near offset med freq - Far offset med freq = %.1f Hz" %(med1-med2))

plt.plot(xarray,spectrum1,color='b',linewidth=2,label="Near offsets")
plt.plot(xarray,spectrum2,color='r',linewidth=1,label="Far offsets")
plt.vlines(med1,0,ymax,colors='b',linestyles='--',linewidth=0.5)
plt.vlines(med2,0,ymax,colors='r',linestyles='--',linewidth=0.5)
plt.gca().set_yticks([])
plt.xlabel('Frequency (Hz)')
plt.ylabel("Normalized amplitude")
plt.xlim([0,500])
plt.ylim([0,ymax])
plt.grid(color='k', linestyle='-', linewidth=0.2)
plt.gca().xaxis.set_major_formatter(FormatStrFormatter('%.0f'))
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.legend(loc='upper right',prop={'size': 6})
plt.title(r'c) $Q_P=Q_S=60$',loc='left')

plt.subplot(2,2,4)

spectrum1 = np.sum(sf4_fx[0,140:190,:],axis=0)
spectrum2 = np.sum(sf4_fx[0,440:490,:],axis=0)
area1 = np.trapz(spectrum1,dx=df)
area2 = np.trapz(spectrum2,dx=df)
spectrum1 /= area1
spectrum2 /= area2
med1 = xarray[findMed(spectrum1,df)]
med2 = xarray[findMed(spectrum2,df)]
ymax=max(np.max(spectrum1),np.max(spectrum2))
print("Near offset med freq - Far offset med freq = %.1f Hz" %(med1-med2))

plt.plot(xarray,spectrum1,color='b',linewidth=2,label="Near offsets")
plt.plot(xarray,spectrum2,color='r',linewidth=1,label="Far offsets")
plt.vlines(med1,0,ymax,colors='b',linestyles='--',linewidth=0.5)
plt.vlines(med2,0,ymax,colors='r',linestyles='--',linewidth=0.5)
plt.gca().set_yticks([])
plt.xlabel('Frequency (Hz)')
plt.xlim([0,500])
plt.ylim([0,ymax])
plt.grid(color='k', linestyle='-', linewidth=0.2)
plt.gca().xaxis.set_major_formatter(FormatStrFormatter('%.0f'))
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.legend(loc='upper right',prop={'size': 6})
plt.title(r'd) $Q_P=Q_S=30$',loc='left')


plt.savefig(figpath+'synthetic_spectra.png',bbox_inches='tight',format='png')

In [ ]:
# DAS data average spectra

istart=10 # skip frequencies ~< 30 Hz
nf=axes_fx.n[0] - istart
df=axes_fx.d[0]
fmin=axes_fx.o[0] + istart*df
fmax=df*(nf-1) + fmin

xarray = np.linspace(fmin,(nf-1)*df+fmin,nf)
data = np.sum(das_fx[2:,:,istart:],axis=0) # sum the FX of 14 shots from the stimulated side

plt.figure(figsize=(3.33, 3.33),dpi=300)

spectrum1 = np.sum(data[0:50,:],axis=0) # the first offset is 150 m
spectrum2 = np.sum(data[300:350,:],axis=0)
area1 = np.trapz(spectrum1,dx=df)
area2 = np.trapz(spectrum2,dx=df)
spectrum1 /= area1
spectrum2 /= area2
med1 = xarray[findMed(spectrum1,df)]
med2 = xarray[findMed(spectrum2,df)]
ymax=max(np.max(spectrum1),np.max(spectrum2))
print("Near offset med freq - Far offset med freq = %.1f Hz" %(med1-med2))

plt.plot(xarray,spectrum1,color='b',linewidth=2,label="Near offsets")
plt.plot(xarray,spectrum2,color='r',linewidth=1,label="Far offsets")
plt.vlines(med1,0,ymax,colors='b',linestyles='--',linewidth=0.5)
plt.vlines(med2,0,ymax,colors='r',linestyles='--',linewidth=0.5)
plt.xlabel('Frequency (Hz)')
plt.ylabel("Normalized amplitude")
plt.gca().set_yticks([])
plt.xlim([fmin,fmax])
plt.ylim([0,ymax])
plt.grid(color='k', linestyle='-', linewidth=0.2)
plt.gca().xaxis.set_major_formatter(FormatStrFormatter('%.0f'))
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.legend(loc='upper right',prop={'size': 6})

plt.savefig(figpath+'das_spectra.png',bbox_inches='tight',format='png')